In [1]:
import numpy as np
import torch

In [5]:
import os
import datetime
import logging
import glob
import shutil

In [14]:
def make_directories():
    '''
    Creates directories for storing data during a model training run
    '''    
    # Get current date for saving folder
    date = datetime.datetime.today().strftime('%Y-%m-%d')
    # Initialise the run and dir_check to create a new run folder within the current date
    run = 0
    dir_check = True
    # Initialise all pahts
    train_path, model_path, save_path, script_path, run_path = None, None, None, None, None
    # Find the current run: the first run that doesn't exist yet
    while dir_check:
        # Construct new paths
        run_path    = f'./Summaries/{date}/run{run}/'
        train_path  = os.path.join(run_path, 'train')
        model_path  = os.path.join(run_path, 'model')
        save_path   = os.path.join(run_path, 'save')
        script_path = os.path.join(run_path, 'script')
        envs_path   = os.path.join(script_path, 'envs')
        run += 1

        # And once a path doesn't exist yet: create new folders
        if not os.path.exists(train_path) and not os.path.exists(model_path) and not os.path.exists(save_path):
            os.makedirs(train_path)
            os.makedirs(model_path)
            os.makedirs(save_path)
            os.makedirs(script_path)
            os.makedirs(envs_path)
            dir_check = False
    # Return folders to new path
    return run_path, train_path, model_path, save_path, script_path, envs_path

In [15]:
# Start training from step 0
i_start = 0

# Create directories for storing all information about the current run
run_path, train_path, model_path, save_path, script_path, envs_path = make_directories()

In [16]:
# Save all python files in current directory to script directory
files = glob.iglob(os.path.join('.', '*.py'))
for file in files:
    if os.path.isfile(file):
        dst = os.path.join(script_path, file)
        print(f'copying {file=} to {dst=}')
        shutil.copy2(file, dst) 

copying file='./hello.py' to dst='./Summaries/2025-05-19/run0/script/./hello.py'


In [28]:
def parameters():
    params = {}
    # -- Model parameters   
    # Decide whether to use seperate grid modules that recieve shiny information for object vector cells.
    # To disable OVC, set this False, and set n_ovc to [0 for _ in range(len(params['n_g_subsampled']))].
    params['separate_ovc'] = False

    # ---- Neuron and module parameters
    # Neurons for subsampled entorhinal abstract location f_g(g) for each frequency module
    params['n_g_subsampled'] = [10, 10, 8, 6, 6]
    # Neurons for object vector cells. Neurons will get new modules if object vector cell modules
    # are separated; otherwise, they are added to existing abstract location modules.
    # a) No additional modules, no additional object vector neurons (e.g. when not using shiny
    #    environments): [0 for _ in range(len(params['n_g_subsampled']))], and separate_ovc set to False
    # b) No additional modules, but n additional object vector neurons in each grid module:
    #    [n for _ in range(len(params['n_g_subsampled']))], and separate_ovc set to False
    # c) Additional separate object vector modules, with n, m neurons: [n, m], and separate_ovc set to
    #    True
    params['n_ovc'] = [0 for _ in range(len(params['n_g_subsampled']))]
    # Total number of modules
    params['n_f'] = len(params['n_g_subsampled'])

    # Number of hierarchical frequency modules for object vector cells
    params['n_f_ovc'] = len(params['n_ovc']) if params['separate_ovc'] else 0

    # Initial frequencies of each module. For ease of interpretation (higher number = higher frequency)
    # this is 1 - the frequency as James uses it
    params['f_initial'] = [0.99, 0.3, 0.09, 0.03, 0.01]
    # Add frequencies of object vector cell modules, if object vector cells get separate modules
    params['f_initial'] = params['f_initial'] + params['f_initial'][0:params['n_f_ovc']]
    return params

# Initalise hyperparameters for model
params = parameters()

In [29]:
params

{'separate_ovc': False,
 'n_g_subsampled': [10, 10, 8, 6, 6],
 'n_ovc': [0, 0, 0, 0, 0],
 'n_f': 5,
 'n_f_ovc': 0,
 'f_initial': [0.99, 0.3, 0.09, 0.03, 0.01]}

In [30]:
# Save parameters
np.save(os.path.join(save_path, 'params'), params)

# Model

In [31]:
hyper = params

In [32]:
# Scale factor in Laplacian transform for each frequency module. High frequency comes first, low frequency comes last. Learn inverse sigmoid instead of scale factor directly, so domain of alpha is -inf, inf
alpha = torch.nn.ParameterList(
    [
        torch.nn.Parameter(
            torch.tensor(
                np.log(hyper['f_initial'][f] / (1 - hyper['f_initial'][f])),
                dtype=torch.float,
            )
        ) for f in range(hyper['n_f'])
    ]
)

In [33]:
alpha

ParameterList(
    (0): Parameter containing: [torch.float32 of size ]
    (1): Parameter containing: [torch.float32 of size ]
    (2): Parameter containing: [torch.float32 of size ]
    (3): Parameter containing: [torch.float32 of size ]
    (4): Parameter containing: [torch.float32 of size ]
)